# ML Training Data Generation with Parameter Sweeps

This notebook demonstrates how to generate synthetic training datasets for machine learning by systematically varying model parameters.

**Use cases:**
- Training neural networks for spectroscopy analysis
- Testing parameter recovery algorithms
- Exploring parameter sensitivity and identifiability
- Generating data for uncertainty quantification studies

In [ ]:
import os
import numpy as np
import trspecfit
from trspecfit.utils.sweep import ParameterSweep, SweepDataset

## 1. Set Up Model

First, create your model as usual. This will be the "ground truth" model that we'll vary to generate training data.

In [ ]:
# Create parent project
project = trspecfit.Project(path=os.getcwd())

# Create file instance with axes
file = trspecfit.File(
    parent_project=project,
    energy=np.arange(0, 20, 0.01),
    time=np.arange(-10, 100, 0.1)
)

In [ ]:
# Load energy-resolved model
file.load_model(
    model_yaml='models_energy.yaml',
    model_info=['single_peak']
)

In [ ]:
# Add time dependence to make it 2D
file.add_time_dependence(
    model_yaml="models_time.yaml",
    model_info=['MonoExpPosIRF'],
    par_name="GLP_01_x0"
)

In [ ]:
# Examine the model structure
file.model_active.describe()

## 2. Define Parameter Space

Choose which parameters to vary and how. You can use:
- **Discrete ranges**: `add_range()` for specific values
- **Continuous distributions**: `add_uniform()`, `add_normal()`, `add_lognormal()`

Strategy options:
- **`'grid'`**: Full combinatorial - every combination (good for small spaces)
- **`'random'`**: Independent sampling (scalable for ML)

### Example 1: Small Grid Sweep

Good for systematic exploration of a few parameter values.

In [ ]:
# Define a small grid
sweep_grid = ParameterSweep(strategy='grid', seed=42)
sweep_grid.add_range('GLP_01_x0_expFun_01_A', [0.25, 1, 4])
sweep_grid.add_uniform('GLP_01_x0_expFun_01_tau', min_val=5, max_val=25, n_samples=3)

print(f"Total configurations: {sweep_grid.get_n_configs()}")
print("\nFirst 5 configurations:")
for i, config in enumerate(sweep_grid):
    if i >= 5:
        break
    print(f"  Config {i}: {config}")

### Example 2: Random Sampling (Recommended for ML)

Samples parameters independently - scales to large datasets.

In [ ]:
# Define random sampling sweep
sweep_random = ParameterSweep(strategy='random', seed=42)

# Vary peak amplitude
sweep_random.add_uniform('GLP_01_A', min_val=8, max_val=12, n_samples=100)

# Vary peak position
sweep_random.add_uniform('GLP_01_x0', min_val=8, max_val=12, n_samples=100)

# Vary dynamics amplitude
sweep_random.add_uniform('GLP_01_x0_expFun_01_A', min_val=0.5, max_val=5, n_samples=100)

# Vary decay time constant
sweep_random.add_uniform('GLP_01_x0_expFun_01_tau', min_val=10, max_val=80, n_samples=100)

print(f"Total configurations: {sweep_random.get_n_configs()}")
print("\nFirst 5 configurations:")
for i, config in enumerate(sweep_random):
    if i >= 5:
        break
    print(f"  Config {i}:")
    for k, v in config.items():
        print(f"    {k}: {v:.3f}")

## 3. Generate Training Data

Use `simulate_parameter_sweep()` to generate datasets. This:
1. Iterates through parameter configurations
2. Updates model parameters
3. Generates N noisy realizations per configuration
4. Saves everything to HDF5 file

**Memory efficiency**: Only one configuration in memory at a time!

In [ ]:
# Create simulator
sim = trspecfit.Simulator(
    model=file.model_active,
    detection='analog',
    noise_level=0.05,
    noise_type='poisson',
    seed=42
)

### Small Test Dataset (Quick)

Generate a small dataset first to verify everything works.

In [ ]:
# Use the grid sweep for quick test
sim.simulate_parameter_sweep(
    parameter_sweep=sweep_grid,
    N_realizations=3,
    filepath='test_sweep.h5',
    show_progress=True
)

### Large Training Dataset (Takes Time)

Generate the full ML training dataset. Adjust N_realizations based on your needs:
- **N=10-20**: Good for most ML applications
- **N=50+**: For robust uncertainty quantification

In [ ]:
# Generate large training dataset
# WARNING: This can take up to several minutes!
sim.simulate_parameter_sweep(
    parameter_sweep=sweep_random,
    N_realizations=20,
    filepath='ml_training_data.h5',
    show_progress=True
)

# Result: 100 configs × 20 realizations = 2000 training examples

## 4. Inspect Generated Data

Use `SweepDataset` to examine what was generated.

In [ ]:
# Load dataset inspector
dataset = SweepDataset('simulated_data/ml_training_data.h5')
#dataset = SweepDataset('simulated_data/test_sweep.h5')

In [ ]:
# Print comprehensive summary
dataset.print_summary()

In [ ]:
# Get parameter summary as DataFrame
df = dataset.get_parameter_summary()
print("\nParameter configurations:")
display(df.round(3))

print("\nStatistical summary:")
display(df.describe().round(3))

In [ ]:
# Visualize parameter distributions
import matplotlib.pyplot as plt

df.hist(figsize=(12, 8), bins=20)
plt.suptitle('Parameter Distributions in Training Data')
plt.tight_layout()
plt.show()

In [ ]:
# Load a specific configuration for inspection
config = dataset.load_config(0)

print(f"Config 0 parameters:")
for k, v in config['parameters'].items():
    print(f"  {k}: {v:.3f}")

print(f"\nData shapes:")
print(f"  Clean: {config['clean'].shape}")
print(f"  Noisy realizations: {len(config['noisy'])}")
print(f"  Each realization: {config['noisy'][0].shape}")

In [ ]:
# Visualize a configuration (uses simulator's built-in plotting)
print("\nVisualize configuration 0:")
print(f"  - Or use your own plotting with dataset.get_axes() for x/y axes")
print(f"  - Or (see below): Load config, set sim.data_clean/noisy, call sim.plot_comparison(dim=2)")
# Temporarily set simulator data to this config for plotting
sim.data_clean = config['clean']
sim.data_noisy = config['noisy'][0]
sim.noise = config['noisy'][0] - config['clean']
sim.plot_comparison(dim=2)

## 5. Loading Data for ML Training

For actual ML training, you'll typically write a custom data loader for your framework (PyTorch, TensorFlow, etc.). Here's a basic example using h5py directly:

In [ ]:
import h5py

# Example: Load all data into memory (for small datasets)
def load_training_data(filepath, param_names):
    """
    Load parameter sweep data for ML training.
    
    Parameters
    ----------
    filepath : str
        Path to HDF5 file
    param_names : list of str
        Which parameters to extract as targets
    
    Returns
    -------
    X : list of arrays
        Input spectra (noisy realizations)
    y : list of arrays
        Target parameters for each spectrum
    """
    X = []  # Input: noisy spectra
    y = []  # Target: parameter values
    
    with h5py.File(filepath, 'r') as f:
        n_configs = f['metadata'].attrs['n_configs']
        
        for config_idx in range(n_configs):
            config_name = f'config_{config_idx:06d}'
            config_group = f['parameter_configs'][config_name]
            data_group = f['simulated_data'][config_name]
            
            # Extract target parameters
            targets = [config_group.attrs[name] for name in param_names]
            
            # Load all noisy realizations for this config
            for real_key in sorted(data_group.keys()):
                spectrum = data_group[real_key][:]
                X.append(spectrum)
                y.append(targets)
    
    return np.array(X), np.array(y)

# Example usage
X_train, y_train = load_training_data(
    'simulated_data/test_sweep.h5',
    param_names=['GLP_01_x0_expFun_01_A', 'GLP_01_x0_expFun_01_tau']
)

print(f"Training data shape: {X_train.shape}")
print(f"Target shape: {y_train.shape}")
print(f"\nFirst example target: {y_train[0]}")

### Example: PyTorch Dataset Class

For large datasets that don't fit in memory:

In [ ]:
# Uncomment if you have PyTorch installed:
# import torch
# from torch.utils.data import Dataset, DataLoader
# 
# class SpectroscopyDataset(Dataset):
#     """PyTorch dataset for parameter sweep data."""
#     
#     def __init__(self, filepath, param_names, transform=None):
#         self.filepath = filepath
#         self.param_names = param_names
#         self.transform = transform
#         
#         # Load metadata
#         with h5py.File(filepath, 'r') as f:
#             self.n_configs = f['metadata'].attrs['n_configs']
#             self.n_realizations = f['metadata'].attrs['n_realizations_per_config']
#         
#         self.total_samples = self.n_configs * self.n_realizations
#     
#     def __len__(self):
#         return self.total_samples
#     
#     def __getitem__(self, idx):
#         # Calculate which config and realization
#         config_idx = idx // self.n_realizations
#         real_idx = idx % self.n_realizations
#         
#         with h5py.File(self.filepath, 'r') as f:
#             config_name = f'config_{config_idx:06d}'
#             config_group = f['parameter_configs'][config_name]
#             data_group = f['simulated_data'][config_name]
#             
#             # Load spectrum
#             spectrum = data_group[f'{real_idx:06d}'][:]
#             
#             # Load target parameters
#             targets = [config_group.attrs[name] for name in self.param_names]
#         
#         if self.transform:
#             spectrum = self.transform(spectrum)
#         
#         return torch.FloatTensor(spectrum), torch.FloatTensor(targets)
# 
# # Create data loader
# dataset = SpectroscopyDataset(
#     'simulated_data/test_sweep.h5',
#     param_names=['GLP_01_x0_expFun_01_A', 'GLP_01_x0_expFun_01_tau']
# )
# 
# train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
# 
# # Use in training loop
# for batch_spectra, batch_targets in train_loader:
#     # ... training code ...
#     pass

print("See commented code above for PyTorch example")

## Tips for ML Training Data

**Parameter Selection:**
- Start with 2-3 most important parameters
- Use realistic ranges from your experimental data
- Include correlations if parameters are physically related

**Dataset Size:**
- Small networks: 1,000-5,000 examples often sufficient
- Larger networks: 10,000-50,000 examples
- More realizations per config = better uncertainty estimates

**Noise Level:**
- Match your experimental SNR
- Can generate multiple datasets with different noise levels
- Consider augmentation during training (add more noise)

**Validation Strategy:**
- Split by configuration, not by realization
- Test on held-out parameter ranges
- Validate on real experimental data when possible